In [1]:
! pip install datasets
! pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 12.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares

In [3]:
from datasets import load_dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "5core_last_out_w_his_All_Beauty", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

All_Beauty.train.csv:   0%|          | 0.00/274k [00:00<?, ?B/s]

All_Beauty.valid.csv:   0%|          | 0.00/37.1k [00:00<?, ?B/s]

All_Beauty.test.csv:   0%|          | 0.00/39.9k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)

All_Beauty.jsonl:   0%|          | 0.00/327M [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

In [5]:
item_meta = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full", trust_remote_code=True)

meta_All_Beauty.jsonl:   0%|          | 0.00/213M [00:00<?, ?B/s]

Generating full split:   0%|          | 0/112590 [00:00<?, ? examples/s]

In [6]:
ratings = []
for i in range(len(dataset['train'])):
    ratings.append({
        'user_id': dataset['train'][i]['user_id'],
        'parent_asin': dataset['train'][i]['parent_asin'],
        'rating': float(dataset['train'][i]['rating'])
    })

for j in range(len(dataset['valid'])):
    ratings.append({
        'user_id': dataset['valid'][j]['user_id'],
        'parent_asin': dataset['valid'][j]['parent_asin'],
        'rating': float(dataset['valid'][j]['rating'])
    })

In [7]:
ratings_df = pd.DataFrame(ratings)

In [8]:
ratings_df

,user_id,parent_asin,rating
0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07J3GH1W1,5.0
1,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07W397QG4,5.0
2,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07KG1TWP5,5.0
3,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08JTNQFZY,5.0
4,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07SLFWZKN,3.0
...,...,...,...
2277,AEOTWG667CGEIHYH33COIX53Y25Q,B00O2FGBJS,5.0
2278,AHT556AWAYQIXS3RB57QNRQLBARQ,B08QFGV5ZQ,4.0
2279,AESI2BA4YODTHOSRFLJCSTAM6XDQ,B089NJG212,3.0
2280,AFKZESU3PTCQ2UVDBDSFVFTXBFNQ_2,B07RM722DH,3.0


In [9]:
item_meta_info = []
pa_uniq = ratings_df['parent_asin'].unique()
for i in range(len(item_meta)):
    if item_meta[i]['parent_asin'] in pa_uniq:
      item_meta_info.append({
          'parent_asin':item_meta[i]['parent_asin'],
          'average_rating':item_meta[i]['average_rating'],
          'rating_number':item_meta[i]['rating_number'],
          'store': item_meta[i]['store'],

      })

In [10]:
item_meta_df = pd.DataFrame(item_meta_info)

In [11]:
# Convert user and product IDs to integer indices for the matrix
user_mapping = {user: idx for idx, user in enumerate(ratings_df['user_id'].unique())}
product_mapping = {product: idx for idx, product in enumerate(ratings_df['parent_asin'].unique())}
reverse_user_mapping = {idx: user for user, idx in user_mapping.items()}
reverse_product_mapping = {idx: product for product, idx in product_mapping.items()}

In [12]:
# convert to scipy sparse matrix
ratings_df['user_id'] = ratings_df['user_id'].map(user_mapping)
ratings_df['parent_asin'] = ratings_df['parent_asin'].map(product_mapping)

In [13]:
# Create the sparse matrix
user_item = coo_matrix(
    (ratings_df['rating'], (ratings_df['user_id'], ratings_df['parent_asin']))
).tocsr()

In [14]:
# Train the ALS model
model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Train the model
model.fit(user_item)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
user_ids = np.arange(user_item.shape[0])
item_ids, scores = model.recommend(user_ids, user_item[user_ids], N=20)

def map_value(x):
    return reverse_product_mapping.get(x)

vectorized_map = np.vectorize(map_value)
recommended_products = vectorized_map(item_ids)

In [16]:
ground_truth = [x['parent_asin'] for x in dataset['test']]

In [17]:
def evaluate_recommendations(recommendations, ground_truth):
    """
    Evaluate Hit Rate and Recall for the recommendations.
    """
    hits = 0
    recall_sum = 0
    total_relevant_items = 0

    for i, actual_items in enumerate(ground_truth):
        recommended_items = recommendations[i]
        actual_set = set([actual_items])

        # Calculate hits (whether there's any overlap)
        if any(item in actual_set for item in recommended_items):
            hits += 1

        # Calculate recall (proportion of relevant items in recommendations)
        relevant_items = actual_set.intersection(recommended_items)
        recall_sum += len(relevant_items)
        total_relevant_items += len(actual_set)

    # Hit Rate: Proportion of users with at least one hit
    hit_rate = hits / len(ground_truth)

    # Recall: Average recall across all users
    recall = recall_sum / total_relevant_items

    return {'hit_rate': hit_rate, 'recall': recall}

In [18]:
results = evaluate_recommendations(recommended_products, ground_truth)
print('hit rate is', results['hit_rate'])
print('recall is', results['recall'])

hit rate is 0.07509881422924901
recall is 0.07509881422924901


# neural with side info

In [32]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

scaler = MinMaxScaler()
item_meta_df['average_rating'] = scaler.fit_transform(item_meta_df[['average_rating']])
item_meta_df['rating_number'] = scaler.fit_transform(item_meta_df[['rating_number']])

label_encoder = LabelEncoder()
item_meta_df['store_encoded'] = label_encoder.fit_transform(item_meta_df['store'])
item_meta_mapping = item_meta_df.set_index('parent_asin').to_dict(orient='index')

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

num_users = ratings_df['user_id'].nunique()
num_items = ratings_df['parent_asin'].nunique()
num_stores = item_meta_df['store_encoded'].nunique()


# Input layers
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
average_rating_input = Input(shape=(1,), name='average_rating_input')  # Average rating
rating_number_input = Input(shape=(1,), name='rating_number_input')  # Rating count
#store_input = Input(shape=(1,), name='store_input')  # Categorical store feature


# Embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=16)(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=16)(item_input)
#store_embedding = Embedding(input_dim=num_stores, output_dim=4)(store_input)

# Flatten and concatenate
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)
#store_vec = Flatten()(store_embedding)

item_combined = Concatenate()([item_vec, average_rating_input, rating_number_input])

combined = Concatenate()([user_vec, item_combined])

# Fully connected layers
dense_1 = Dense(64, activation='relu')(combined)
dense_2 = Dense(32, activation='relu')(dense_1)
output = Dense(1)(dense_2)

# Build and compile the model
model = Model(inputs=[user_input, item_input, average_rating_input, rating_number_input], outputs=output)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

ratings_df['average_rating'] = pd.DataFrame(ratings)['parent_asin'].map(lambda x: item_meta_mapping[x]['average_rating'])
ratings_df['rating_number'] = pd.DataFrame(ratings)['parent_asin'].map(lambda x: item_meta_mapping[x]['rating_number'])
#ratings_df['store_encoded'] = pd.DataFrame(ratings)['parent_asin'].map(lambda x: item_meta_mapping[x]['store_encoded'])

# Prepare input data
X = [
    ratings_df['user_id'],                # User IDs
    ratings_df['parent_asin'],            # Item IDs
    ratings_df['average_rating'],         # Average ratings (numerical)
    ratings_df['rating_number'],          # Rating count (numerical)
   # ratings_df['store_encoded']           # Store (categorical)
]

# Train the model
model.fit(X, ratings_df['rating'], epochs=16, batch_size=64)


Epoch 1/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 18.5544 - mae: 4.2027
Epoch 2/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.5155 - mae: 2.8937
Epoch 3/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9726 - mae: 0.7586
Epoch 4/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6328 - mae: 0.6254
Epoch 5/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5751 - mae: 0.5719
Epoch 6/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5289 - mae: 0.5486
Epoch 7/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5258 - mae: 0.5456
Epoch 8/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.5006 - mae: 0.5233
Epoch 9/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5052 - mae: 0.5315
Epoch 10/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5066 - mae: 0.5323
Epoch 11/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4983 - mae: 0.5274
Epoch 12/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4978 - mae: 0.5327
Epoch 13/16
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

In [53]:
import numpy as np

# Get all unique user and item IDs
user_ids = np.arange(user_item.shape[0])
item_ids = pd.DataFrame(ratings)['parent_asin'].unique()
# Create mappings back to product IDs

average_ratings_test = np.array([item_meta_mapping[item]['average_rating'] for item in item_meta_df['parent_asin']])
rating_numbers_test = np.array([item_meta_mapping[item]['rating_number'] for item in item_meta_df['parent_asin']])
#stores_test = np.array([item_meta_mapping[item]['store_encoded'] for item in item_meta_df['parent_asin']])
item_id_to_product = {idx: product for idx, product in enumerate(item_ids)}
product_to_item_id = {product: idx for idx, product in enumerate(item_ids)}

# N - Number of top recommendations
N = 20

# Generate Top-N recommendations for all users
recommended_products_1 = []

# Loop through all user IDs
for user in user_ids:
    # Prepare all user-item pairs for the current user
    user_item_pairs = np.array([[user, i] for i, item_id in enumerate(item_ids)])

    # Extract metadata for the items
    average_ratings = np.array([item_meta_mapping[item]['average_rating'] for item in item_ids])
    rating_numbers = np.array([item_meta_mapping[item]['rating_number'] for item in item_ids])
    #stores = np.array([item_meta_mapping[item]['store_encoded'] for item in item_ids])

    # Predict ratings using the NCF model
    predictions = model.predict([
        user_item_pairs[:, 0],  # User IDs
        user_item_pairs[:, 1],  # Item IDs
        average_ratings,        # Average ratings
        rating_numbers,         # Rating numbers
     #   stores                  # Store categories
    ])

    # Sort items by predicted score and select Top-N
    top_indices = predictions.flatten().argsort()[-N:][::-1]
    top_items = [item_id_to_product[item_idx] for item_idx in top_indices]  # Map to original product IDs

    # Append the recommendations for the current user
    recommended_products_1.append(top_items)

# Convert to NumPy array if needed
recommended_products_1 = np.array(recommended_products_1)





11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [54]:
# Evaluate recommendations
results1 = evaluate_recommendations(recommended_products_1, ground_truth)

# Print the results
print('Hit Rate:', results1['hit_rate'])
print('Recall:', results1['recall'])

Hit Rate: 0.07114624505928854
Recall: 0.07114624505928854


In [45]:
rowwise_union = []
for row1, row2 in zip(recommended_products_1, recommended_products):
    union_items = np.union1d(row1, row2)  # Union for the current pair of rows
    rowwise_union.append(union_items)

# Convert to a NumPy array with dtype=object for variable-length results
rowwise_union = np.array(rowwise_union, dtype=object)


In [46]:
with open('cf_retrival.npy', 'wb') as f:
    np.save(f, recommended_products)

In [47]:
with open('neuralcf_sideinfo_retrival.npy', 'wb') as f:
    np.save(f, recommended_products_1)

In [28]:
with open('combined_retrival.npy', 'wb') as f:
    np.save(f, rowwise_union)

In [29]:
import pickle
with open('cf_retrieval.pkl', 'wb') as f:
    pickle.dump(recommended_products, f)

In [30]:
with open('neuralcf_sideinfo_retrival.pkl', 'wb') as f:
    pickle.dump(recommended_products_1, f)

In [31]:
with open('combined_retrival.pkl', 'wb') as f:
    pickle.dump(rowwise_union, f)